<a href="https://colab.research.google.com/github/vicotrbb/data_science/blob/master/projects/checkers/checkers_artificial_intelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Simple checkers artificial intelligence game agent

In [ ]:
from IPython.display import clear_output
import sys
from copy import deepcopy
from operator import itemgetter
from time import sleep
from random import randint

In [ ]:
# 0 = campo vazio
# 1 = peça preta
# 2 = peça branca
# 3 = dama preta
# 4 = dama branca

def generate_board():
  """
    Gera um tabuleiro de damas com as pessoas em suas respectivas posições
  """
  board = [[0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0]]

  # posiciona as peças pretas:
  board[0][0], board[0][2], board[0][4], board[0][6] = 1, 1, 1, 1
  board[1][1], board[1][3], board[1][5], board[1][7] = 1, 1, 1, 1
  board[2][0], board[2][2], board[2][4], board[2][6] = 1, 1, 1, 1

  # posiciona as peças brancas:
  board[5][1], board[5][3], board[5][5], board[5][7] = 2, 2, 2, 2
  board[6][0], board[6][2], board[6][4], board[6][6] = 2, 2, 2, 2
  board[7][1], board[7][3], board[7][5], board[7][7] = 2, 2, 2, 2

  return board

In [ ]:
def render_board(board):
  """
    Limpa o output e renderiza o tabuleiro.
  """
  # Limpa o output para permitir a mudança do tabuleiro sem quebra de linha
  clear_output()
  for row in board:
    print(row, end='\n')
  print()

In [ ]:
def map_pieces(player, board):
  """
    Mapeia todas as posições de todas as peças de algum jogador
  """
  piece_type = get_player_wide_pieces(player)
  pieces = []
  for x, board_row in enumerate(board):
    for y, value in enumerate(board_row):
      if value in piece_type: pieces.append((x, y))
  return pieces

In [ ]:
def get_opposite_player_piece(player):
  """
    Retorna o jogador adversário da peça em questão
  """
  map = {
      1: 2,
      2: 1,
      3: 2,
      4: 1
  }
  return map[player]

In [ ]:
def get_player_wide_pieces(player):
  """
    Retorna o set de peças possiveis para um jogador
  """
  if player == 1: return [1, 3]
  if player == 2: return [2, 4]

In [ ]:
def check_if_player_won(player, board):
  """
    Verifica se um jogador em especial ganhou
  """
  other_player = get_opposite_player_piece(player)
  for x, board_row in enumerate(board):
    for y, value in enumerate(board_row):
      if value == other_player: return False
  return True

In [ ]:
def check_if_the_game_finished(board):
  """
    Verifica se o jogo terminou e quem ganhou
  """
  if check_if_player_won(1, board): return 'Pretas ganharam'
  if check_if_player_won(2, board): return 'Brancas ganharam' 
  return 'jogando'

In [ ]:
def evaluate_possible_moves(position, board):
  """
    Avalia quais as melhores opções de jogadas disponiveis para uma peça em
    questão, a avaliação é deterministica e busca apenas verificar as jogadas
    possiveis de acordo com a posição da peça, tipo de peça, e se os movimentos
    são validos (não permite movimentos em cima de peças amigas). Movimentos 
    de 'comer' o adversário são priorizados e colocados a frente da lista de 
    movimentos.
  """
  x, y = position

  if board[x][y] == 0:
    return

  checker = lambda x,y: x + y >= 0 and x + y < 8
  positions = []
  vectors = [[1, -1], [1, 1]] if board[x][y] == 1 else [[-1, -1], [-1, 1]]

  if board[x][y] in [3, 4]:
    vectors = [[1, -1], [1, 1], [-1, -1], [-1, 1]]

  for vector in vectors:
    x_v, y_v = vector

    if checker(x_v, x) and checker(y_v, y):
      if not board[(x + x_v)][(y + y_v)]:
        positions.append((x + x_v, y + y_v))

      elif board[x + x_v][y + y_v] and board[x + x_v][y + y_v] != board[x][y]:
        if checker((2 * x_v), x) and checker((2 * y_v), y) and not board[(2 * x_v)+ x][(2 * y_v) + y]:
            positions.insert(0, (x_v + x, y_v + y ))
 
  return positions

In [ ]:
def perform_the_next_optimal_value(player, board, depth=5, randomness_treshhold=3):
  """
    Analisa qual a melhor jogada para um jogador em dado momento, realizado uma
    previsão do futuro baseado em profunidade de jogadas e iteração em profundidade.
    
    Basicamente, o algoritmo pega as jogadas possiveis para cada peça, executa 
    essa jogada de forma simulada n vezes (n = depth) e atribui pontos para a 
    jogada inicial, ao final, o algoritmo irá selecionar randomicamente uma das 
    melhores jogadas.
    
    A escolha de forma randomica é importante para balencear o determinismo da
    escolha da melhor jogada, visto que é baseada em pontos e o ambiente onde o 
    algoritmo atua é estatico no estado inicial.
  """
  MAX_VALUE = float('inf')
  MEAN_VALUE = 99
  player_wide_possible_moves = [] # dict(piece, piece_move, move_points)
  opposite = get_opposite_player_piece(player)

  pieces = map_pieces(player, board)
  for piece in pieces:
    possible_piece_moves = []
    initial_possible_moves = evaluate_possible_moves(piece, board)
    
    for move in initial_possible_moves:
      move_points = 0
      new_board = deepcopy(board)
      new_move = deepcopy(move)

      for _ in range(depth):
        if new_move[0] == 7 and player == 1:
          move_points += MEAN_VALUE
        elif new_move[0] == 0 and player == 2:
          move_points += MEAN_VALUE
        else:
          move_points += 1

        new_board[new_move[0]][new_move[1]] = player

        # Em casos reais, esta parte do código deveria ser substituida por uma
        # recursão ou iteração a dentro da profundidade com todas as
        # possibilidades de movimento, entretanto, para facilitar a compreensão
        # e a resolução, iteramos apenas os movimentos iniciais e os movimentos
        # em profundidade foram substituidos para usarmos apenas o primeiro mais
        # otimizado, sem realizar a recursão em profundidade.
        new_move = evaluate_possible_moves(new_move, new_board)
        if not new_move:
          break
        else:
          new_move = new_move[0]

        if new_board[new_move[0]][new_move[1]] == opposite:
          move_points += MEAN_VALUE
          break

        if check_if_player_won(player, new_board):
          possible_piece_moves.append({'piece': piece, 'move': move, 'points': MAX_VALUE})
          break
        
      possible_piece_moves.append({'piece': piece, 'move': move, 'points': move_points})

    if len(possible_piece_moves) != 0:
      designated_move = sorted(possible_piece_moves, key=itemgetter('points'), reverse=True)[0]
      player_wide_possible_moves.append(designated_move)
  
  if player_wide_possible_moves:
    # Execute a randomização da melhor jogada dentro das :randomness_treshhold 
    # melhores.
    result = sorted(player_wide_possible_moves, key=itemgetter('points'), reverse=True)[:randomness_treshhold]
    if len(result) == 1:
      return result[0]
    return result[randint(1, len(result) - 1)]
  return None

In [ ]:
def play_the_game_till_win(board, depth=5):
  """
    Execute e move as peças até que o jogo seja finalizado.
  """
  while check_if_the_game_finished(board) == 'jogando':
    for player in [2, 1]:
      move = perform_the_next_optimal_value(player, board, depth)

      if move:
        move_pos = move['move']
        old_pos = move['piece']
        board[move_pos[0]][move_pos[1]] = player if board[old_pos[0]][old_pos[1]] == player else get_player_wide_pieces(player)[1]
        board[old_pos[0]][old_pos[1]] = 0

        if move_pos[0] == 7 and player == 1:
          board[move_pos[0]][move_pos[1]] = 3
        elif move_pos[0] == 0 and player == 2:
          board[move_pos[0]][move_pos[1]] = 4

      render_board(board)
      sleep(0.5)
      
  render_board(board)
  print('Jogo finalizado')
  print(check_if_the_game_finished(board))

In [ ]:
board = generate_board()
board

[[1, 0, 1, 0, 1, 0, 1, 0],
 [0, 1, 0, 1, 0, 1, 0, 1],
 [1, 0, 1, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 2, 0, 2, 0, 2],
 [2, 0, 2, 0, 2, 0, 2, 0],
 [0, 2, 0, 2, 0, 2, 0, 2]]

In [ ]:
play_the_game_till_win(board, 5)

[4, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 3, 0, 3, 0, 3, 0, 0]

Jogo finalizado
Pretas ganharam
